In [23]:
#import necessary libraries
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import BDay
import random
import math
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from scipy.stats import norm
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')

First lets establish a prophet model and use it to predict the market

In [24]:
# We are analyzing the S&P 500 Index thru the prophet API,
# thus we need to garner the relevant data using yfinance

today = datetime.today().strftime('%Y-%m-%d')
start = '2016-01-01'

sp_df = yf.download('^GSPC', start, today)
sp_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,2038.199951,2038.199951,1989.680054,2012.660034,2012.660034,4304880000
2016-01-05,2013.780029,2021.939941,2004.170044,2016.709961,2016.709961,3706620000
2016-01-06,2011.709961,2011.709961,1979.050049,1990.260010,1990.260010,4336660000
2016-01-07,1985.319946,1985.319946,1938.829956,1943.089966,1943.089966,5076590000
2016-01-08,1945.969971,1960.400024,1918.459961,1922.030029,1922.030029,4664940000
...,...,...,...,...,...,...
2022-09-01,3936.729980,3970.229980,3903.649902,3966.850098,3966.850098,3754570000
2022-09-02,3994.659912,4018.429932,3906.209961,3924.260010,3924.260010,4134920000
2022-09-06,3930.889893,3942.550049,3886.750000,3908.189941,3908.189941,2209800080


In [25]:
# data checks
sp_df.info()
sp_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1683 entries, 2016-01-04 to 2022-09-08
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1683 non-null   float64
 1   High       1683 non-null   float64
 2   Low        1683 non-null   float64
 3   Close      1683 non-null   float64
 4   Adj Close  1683 non-null   float64
 5   Volume     1683 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 92.0 KB


Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [26]:
# prophet is a time series model so we must index by date
sp_df.reset_index(inplace = True)
sp_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [27]:
# prophet models require soley a date and y component
# here that y will be Adj Close

df = sp_df[['Date', 'Adj Close']]

# rename for prophet usage
df.rename(columns = {'Date' : 'ds', 'Adj Close' : 'y'}, inplace = True)
df

,ds,y
0,2016-01-04,2012.660034
1,2016-01-05,2016.709961
2,2016-01-06,1990.260010
3,2016-01-07,1943.089966
4,2016-01-08,1922.030029
...,...,...
1678,2022-09-01,3966.850098
1679,2022-09-02,3924.260010
1680,2022-09-06,3908.189941
1681,2022-09-07,3979.870117


In [28]:
# quickly vizualizing data before prophet use
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = df['ds'], y = df['y']))

fig1.update_layout(
    title_text = "S&P 500 Pricing with Prophet"
)

# adding sliders and buttons for interaction
fig1.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig1.show()

In [29]:
# build prophet model for price forcasting
m = Prophet(daily_seasonality = False, seasonality_mode='multiplicative')
for col in df.columns:
    if col not in ['ds', 'y']:
        m.add_regressor(col, mode = 'additive')
m.fit(df)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [30]:
# here we create a year's worth of time series for the prophet model
future = m.make_future_dataframe(periods = 365)
business_days = BDay().onOffset
filter = pd.to_datetime(future['ds']).map(business_days)
future = future[filter]
future.tail(10)

,ds
2036,2023-08-28
2037,2023-08-29
2038,2023-08-30
2039,2023-08-31
2040,2023-09-01
2043,2023-09-04
2044,2023-09-05
2045,2023-09-06
2046,2023-09-07
2047,2023-09-08


In [31]:
# finally, we get to our predictions
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)

,ds,yhat,yhat_lower,yhat_upper
1934,2023-08-28,3903.464555,1397.030833,6553.328120
1935,2023-08-29,3903.643959,1408.645685,6602.961966
1936,2023-08-30,3905.389388,1379.283328,6597.544293
1937,2023-08-31,3903.283679,1369.707062,6639.657537
1938,2023-09-01,3904.679184,1417.528188,6618.882344
1939,2023-09-04,3902.277923,1380.657305,6681.623863
1940,2023-09-05,3903.215210,1366.006303,6651.223470
1941,2023-09-06,3905.809573,1351.614222,6692.923066
1942,2023-09-07,3904.610430,1338.099282,6726.330232
1943,2023-09-08,3906.919972,1291.240690,6702.987732


In [32]:
# lets specifically get the next days worth!
next_day = ((datetime.today() + timedelta(1)).strftime('%Y-%m-%d'))
forecast[forecast['ds'] == next_day]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat


In [33]:
plot_plotly(m, forecast)

Now, lets approach this problem with a more traditional/basic method in finance: Monte Carlo Simulation

In [34]:
data = pd.DataFrame()
data = sp_df['Adj Close']
data

0       2012.660034
1       2016.709961
2       1990.260010
3       1943.089966
4       1922.030029
           ...     
1678    3966.850098
1679    3924.260010
1680    3908.189941
1681    3979.870117
1682    4006.179932
Name: Adj Close, Length: 1683, dtype: float64

In [35]:
log_returns = np.log(1 + data.pct_change())
log_returns

0            NaN
1       0.002010
2      -0.013202
3      -0.023986
4      -0.010898
          ...   
1678    0.002992
1679   -0.010795
1680   -0.004103
1681    0.018175
1682    0.006589
Name: Adj Close, Length: 1683, dtype: float64

In [36]:
#returns stats
u = log_returns.mean()
var = log_returns.var()
drift = u - (0.5 * var)
stdev = log_returns.std()

# trading days per year and num of sims
t_intervals = 252
iterations = 10

daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))

s0 = data.iloc[-1]
s0

4006.179931640625

In [37]:
price_list = np.zeros_like(daily_returns)
price_list[0] = s0
price_list

array([[4006.17993164, 4006.17993164, 4006.17993164, ..., 4006.17993164,
        4006.17993164, 4006.17993164],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]])

In [38]:
for t in range(1, t_intervals):
    price_list[t] = price_list[t-1] * daily_returns[t]

price_list

columns = []
for i in range(0,10):
     columns.append("sim" + str(i))

df1 = pd.DataFrame(price_list, columns = columns)
df1['date'] = pd.date_range(start= today, periods=len(df1), freq=BDay())
df1

,sim0,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,date
0,4006.179932,4006.179932,4006.179932,4006.179932,4006.179932,4006.179932,4006.179932,4006.179932,4006.179932,4006.179932,2022-09-09
1,4018.021390,4037.261944,4061.554861,4008.725342,4054.575312,3955.477068,4019.709699,4086.563196,3997.964904,4044.441063,2022-09-12
2,4018.095985,3985.891248,4069.501555,3942.924619,4045.146651,3957.398518,4096.030830,4006.315520,4002.445665,4140.562947,2022-09-13
3,3985.087066,3978.618875,4056.762753,4003.374499,4022.531793,3998.470821,4117.742437,3967.534075,3941.809944,4116.309121,2022-09-14
4,3915.027065,3931.177423,4054.172404,3977.149952,3973.419187,4024.470147,4008.713974,3993.462595,3902.492124,4165.186201,2022-09-15
...,...,...,...,...,...,...,...,...,...,...,...
247,3785.368313,3662.643031,2898.184094,6244.548046,6360.706595,3814.847882,3510.540402,4131.587806,3627.467405,3455.845916,2023-08-22
248,3846.007527,3632.379445,2934.745302,6311.993659,6304.003171,3762.912874,3475.899149,4069.821722,3607.953281,3453.504113,2023-08-23
249,3850.341985,3688.789959,2932.635622,6352.682821,6402.144293,3751.457494,3507.377100,4065.046429,3656.640852,3429.452123,2023-08-24
250,3918.317486,3679.400342,2944.487945,6318.919880,6327.531327,3673.999763,3569.517605,4002.788790,3699.696804,3447.346985,2023-08-25


In [39]:
# Let's visualize the Monte Carlo Simulations
fig2 = go.Figure()
for name in columns:
    fig2.add_trace(go.Scatter(x = df1['date'], y = df1[name]))

fig2.update_layout(
    title_text = "S&P 500 Pricing with Monte Carlo"
)

# adding sliders and buttons for interaction
fig2.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig2.show()

For another comparison, lets approach Monte Carlo in a more nuanced way: Euler Discretization

In [40]:
# annual stdev of returns
stdev_a = log_returns.std() * 252 ** 0.5

# risk-free rate of return (bonds)
r = 0.026

# time horizon (years)
T = 1

# intervals within horizon (trading days)
t_intervals = 252
delta_t = T / t_intervals

In [41]:
Z = np.random.standard_normal((t_intervals + 1, iterations))
S = np.zeros_like(Z)
S0 = data.iloc[-1]
S[0] = S0
S

array([[4006.17993164, 4006.17993164, 4006.17993164, ..., 4006.17993164,
        4006.17993164, 4006.17993164],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]])

We need to use the following recursive formula for simulation:
$$
S_t = S_{t-1} \cdot exp((r - 0.5 \cdot stdev^2) \cdot delta_t + stdev \cdot delta_t^{0.5} \cdot Z_t)
$$

In [42]:
for t in range(1, t_intervals + 1):
    S[t] = S[t-1] * np.exp((r - 0.5 * stdev_a ** 2) * delta_t + stdev_a * delta_t ** 0.5 * Z[t])
S

array([[   4006.17993164,    4006.17993164,    4006.17993164, ...,
           4006.17993164,    4006.17993164,    4006.17993164],
       [   4066.37935606,    4102.42660651,    4069.50557796, ...,
           4031.57619347,    4126.23531075,    4091.53156643],
       [   4123.51017601,    4259.47337985,    4201.43149702, ...,
           4129.27160701,    4189.50583325,    4200.54434483],
       ...,
       [2312737.29513802, 3354911.2557868 , 2921383.92379492, ...,
        1670345.59260421, 2458352.18635168, 2195032.01255225],
       [2362487.4115346 , 3378542.48953661, 2939781.61740626, ...,
        1711331.52068426, 2546860.76905951, 2258623.75046804],
       [2402822.17643912, 3512919.77591125, 2990559.69142655, ...,
        1739869.06862204, 2619338.5187665 , 2304676.42772014]])

In [43]:
df2 = pd.DataFrame(S, columns = columns)
df2['date'] = pd.date_range(start= today, periods=len(df2), freq=BDay())
df2

,sim0,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,date
0,4.006180e+03,4.006180e+03,4.006180e+03,4.006180e+03,4.006180e+03,4.006180e+03,4.006180e+03,4.006180e+03,4.006180e+03,4.006180e+03,2022-09-09
1,4.066379e+03,4.102427e+03,4.069506e+03,4.110581e+03,4.079447e+03,4.119706e+03,4.144125e+03,4.031576e+03,4.126235e+03,4.091532e+03,2022-09-12
2,4.123510e+03,4.259473e+03,4.201431e+03,4.212631e+03,4.140489e+03,4.163667e+03,4.258339e+03,4.129272e+03,4.189506e+03,4.200544e+03,2022-09-13
3,4.143550e+03,4.435877e+03,4.349313e+03,4.391103e+03,4.213852e+03,4.252428e+03,4.355504e+03,4.316454e+03,4.281335e+03,4.318237e+03,2022-09-14
4,4.242839e+03,4.620980e+03,4.533207e+03,4.465294e+03,4.388054e+03,4.428880e+03,4.424260e+03,4.368781e+03,4.406939e+03,4.446088e+03,2022-09-15
...,...,...,...,...,...,...,...,...,...,...,...
248,2.165415e+06,3.208017e+06,2.784605e+06,2.031635e+06,1.952211e+06,2.919250e+06,2.605142e+06,1.612291e+06,2.340497e+06,2.038812e+06,2023-08-23
249,2.263175e+06,3.243902e+06,2.860903e+06,2.071427e+06,1.966926e+06,2.980688e+06,2.658717e+06,1.627486e+06,2.404149e+06,2.114913e+06,2023-08-24
250,2.312737e+06,3.354911e+06,2.921384e+06,2.153232e+06,1.997635e+06,3.107737e+06,2.768842e+06,1.670346e+06,2.458352e+06,2.195032e+06,2023-08-25
251,2.362487e+06,3.378542e+06,2.939782e+06,2.236441e+06,2.080336e+06,3.266186e+06,2.889734e+06,1.711332e+06,2.546861e+06,2.258624e+06,2023-08-28


In [44]:
# Let's visualize the Euler Discretizations
fig3 = go.Figure()
for name in columns:
    fig3.add_trace(go.Scatter(x = df2['date'], y = df2[name]))

fig3.update_layout(
    title_text = "S&P 500 Pricing with Euler Discretization"
)

# adding sliders and buttons for interaction
fig3.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig3.show()